In [1]:
import pandas as pd
import glob
import os
import pickle

### System-level data

In [2]:
wmt19_no_metadata = pickle.load(open('../data/pickles/wmt19-sys_level-all.pkl', 'rb'))
wmt19_no_metadata.count()

system       108927
sid          108927
raw_score    108927
score        108927
N            108927
lp           108927
reference    108927
source       108927
output       103501
dtype: int64

In [3]:
write = open('./data/ad-raw.csv', 'wt')
read = open('./data/newstest2019-humaneval/mturk-sntlevel-humaneval-newstest2019/analysis/ad-good-raw.csv', 'rt')

for line in read:
    if '+' not in line:
        write.write(line.replace(' ', '\t'))
    else:
        line = line.split('\t')
        system = line[6]
        for i in system.split('+'):
            l = '\t'.join(line[0:6]) + '\t' + i + '\t' + '\t'.join(line[7:])
            write.write(l)
            #break
write.close()

In [4]:
df = pd.read_csv('./data/ad-raw.csv', sep='\t')
df.count()

HITId         131720
WorkerId      131720
Input.src     131720
Input.trg     131720
Input.item    131720
hit           131720
sys_id        131720
rid           131720
type          131720
sid           131720
score         131720
time          131720
dtype: int64

In [5]:
df['lp'] = df.apply(lambda x: '%s-%s' % (x['Input.src'], x['Input.trg']), axis=1)
df.head(1)

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,lp
0,3MA5N0ATTCBZLJ39L2ULJO4IBBWWK2,M0765,fi,en,ad,1,parfda.6526,0,SYSTEM,367,71,626,fi-en


In [6]:
wmt19_metadata = df.merge(wmt19_no_metadata[['system', 'sid', 'lp', 'reference', 'source', 'output']],
         left_on=['lp', 'sys_id', 'sid'],
         right_on=['lp', 'system', 'sid'],
         how='left'
        )
wmt19_metadata.count()

HITId         131720
WorkerId      131720
Input.src     131720
Input.trg     131720
Input.item    131720
hit           131720
sys_id        131720
rid           131720
type          131720
sid           131720
score         131720
time          131720
lp            131720
system        131720
reference     131720
source        131720
output        131720
dtype: int64

In [7]:
wmt19_metadata[wmt19_metadata.sys_id == 'REFERENCE']

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,lp,system,reference,source,output


In [8]:
wmt19_metadata = wmt19_metadata[['lp', 'HITId', 'WorkerId', 'score', 'time', 'system', 'type', 'sid', 'reference', 'source', 'output']]

In [9]:
pickle.dump(wmt19_metadata, open('pickles/wmt19_sys_metadata.pkl', 'wb'))